In [1]:
{-# LANGUAGE FlexibleContexts, GADTs, TypeOperators #-}
import Data.Manifold.TreeCover
import Data.Manifold.Web
import qualified Data.Graph as Graph
import Data.Manifold.Types
import Data.VectorSpace
import Linear.V2
import Data.AffineSpace
import Math.LinearMap.Category
import Data.Random
import Data.Random.Manifold

import Control.Arrow
import Control.Applicative (empty)
import Data.Foldable (toList, forM_)
import Data.Traversable (forM)

From [dynamic-plot](http://hackage.haskell.org/packages/dynamic-plot):

In [2]:
import Graphics.Dynamic.Plot.R2

From [diagrams](http://projects.haskell.org/diagrams):

In [3]:
import Diagrams.Prelude ( Point(P), r2, circle, (&), (#), (^.), (^&), _1, moveTo
                        , fc, lc, opacity, red, blue, white
                        , fromVertices )
import Diagrams.CubicSpline

Functions for plotting a 2D tree/web structures. Trees with lines for twigs and circles for leaves.

In [4]:
prettyTreePlot :: ShadeTree ℝ² -> [DynamicPlottable]
prettyTreePlot tr = [ plot [ shapePlot $ circle 0.03 & moveTo p & opacity 0.2 | p <- ps ]
                    , plot $ onlyNodes tr ]
                 ++ [unitAspect]
 where ps = map P $ onlyLeaves_ tr
 
prettyWebPlot :: PointsWeb ℝ² () -> [DynamicPlottable]
prettyWebPlot w = [ shapePlot $ cubicSpline False [P r₁, P m, P r₂]
                  | ((r₁@(V2 x₁ y₁),()),(r₂@(V2 x₂ y₂),())) <- edg
                  , let m = V2 ((x₁+2*x₂)/3 + (y₂-y₁)/19)
                               ((y₁+2*y₂)/3 + (x₁-x₂)/19) ]
 where edg = map (gnodes *** gnodes) $ Graph.edges graph
       (graph, gnodes) = toGraph w

plotTreeAndWeb :: ShadeTree ℝ² -> IO ()
plotTreeAndWeb tr = do
    plotWindow $ prettyTreePlot tr # map (tint white)
              ++ prettyWebPlot web # map (tweakPrerendered $ opacity 0.7)
              ++ [plot [ plot [ plot p & tweakPrerendered (opacity 0.3)
                              , shapePlot (circle 0.03 & moveTo (P pp)) ]
                          & tint red
                       | (p@(Cutplane pp _),_) <- webBoundary web ]]
    return ()
 where web = fromShadeTree euclideanMetric tr

### Simple cartesian grid within a disk:

In [5]:
cartTree :: ShadeTree ℝ²
cartTree = fromLeafPoints
              [ (x^&y) | x<-[0, 0.15 .. 4]
                       , y<-[0, 0.2 .. 4]
                       , (x-2)^2 + (y-2)^2 < 4 ]

plotTreeAndWeb cartTree

![visualisation of tree-cover and points-web of cartesian-disk cloud](https://raw.githubusercontent.com/leftaroundabout/manifolds/master/manifolds/images/examples/TreesAndWebs/2D-cartesiandisk.png)

### Regular (slightly stretched) hexagonal-honeycomb grid

In [6]:
honeycombTree = fromLeafPoints $
           [V2 x y | x<-[-2,-1..6], y<-[-1, 0.8..6]]
        ++ [V2 x y | x<-[-1.5,-0.5..6], y<-[-0.7, 1.1 ..6]]
        ++ [V2 x y | x<-[-1.5,-0.5..6], y<-[-0.1, 1.7 ..6]]
        ++ [V2 x y | x<-[-2,-1..6], y<-[0.2, 2.0..6]]

plotTreeAndWeb honeycombTree

![visualisation of tree-cover and points-web of hexagonal points arrangement](https://raw.githubusercontent.com/leftaroundabout/manifolds/master/manifolds/images/examples/TreesAndWebs/2D-hexa-honeycomb.png)

### Regular (slightly stretched) hexagonal grid

In [7]:
hexagonTree = fromLeafPoints $
           [V2 x y | x<-[-2,-1.7..2], y<-[-1,-0.6..2]]
        ++ [V2 x y | x<-[-1.85,-1.55..2], y<-[-0.8,-0.4..2]]

plotTreeAndWeb hexagonTree

![visualisation of tree-cover and points-web of hexagonal points arrangement](https://raw.githubusercontent.com/leftaroundabout/manifolds/master/manifolds/images/examples/TreesAndWebs/2D-hexagonal.png)

### <a id='pseudorandomCloudTree'></a>Cloud of 405 sort-of random points:

In [8]:
scatterTree = let
    tps₀ = [(0,0), (0,1), (1,1), (1,2), (2,2)]
    tps₁ = [p .+^ v^/3 | p<-tps₀, v <- [(0,0), (-1,1), (1,2)]]
    tps₂ = [p .+^ v^/4 | p<-tps₁, v <- [(0,0), (-1,1), (1,2)]]
    tps₃ = [p .+^ v^/5 | p<-tps₂, v <- [(0,0), (-2,1), (1,2)]]
    tps₄ = [p .+^ v^/7 | p<-tps₃, v <- [(0,1), (-1,1), (1,2)]]
     in fromLeafPoints $ r2<$>tps₄
plotTreeAndWeb scatterTree
((_,exampleTwiglet), exampleTwigEnviron) = twigsWithEnvirons scatterTree !! 5
plotWindow $ prettyTreePlot scatterTree
                 # map (tweakPrerendered (opacity 0.3) . tint white)
          ++ prettyTreePlot exampleTwiglet
                 # map (tint blue)
          ++ concat (prettyTreePlot . snd <$> exampleTwigEnviron)
                 # map (tweakPrerendered (opacity 0.3) . tint red)

GraphWindowSpecR2{lBound=-1.8415873015873014, rBound=3.6415873015873013, bBound=-0.5153968253968253, tBound=4.51063492063492, xResolution=640, yResolution=480}

![visualisation of tree-cover and points-wed of pseudorandom-point cloud](https://raw.githubusercontent.com/leftaroundabout/manifolds/master/manifolds/images/examples/TreesAndWebs/2D-scatter.png)
![visualisation of a lowest-level twigs and its neighbours in the tree-cover of a pseudorandom-point cloud](https://raw.githubusercontent.com/leftaroundabout/manifolds/master/manifolds/images/examples/TreesAndWebs/2D-scatter_twig-environs.png)

### 1000 actually-random, normally-distributed points<!--, with focus on one local point-group and its neighbours-->:

In [9]:
randomTr <- fmap fromLeafPoints . forM [0..1000] $ \_->
     runRVar (sample $ (1^&1):±[1^&0, 0^&1]) StdRandom :: IO ℝ²

plotTreeAndWeb randomTr

![visualisation of tree-cover and points-web of pseudorandom-point cloud](https://raw.githubusercontent.com/leftaroundabout/manifolds/master/manifolds/images/examples/TreesAndWebs/2D-normaldistrib.png)